# Speech Regognition with Keras

In [2]:
import os
import librosa
import tensorflow as tf
from IPython.display import Audio, display,update_display
import numpy as np

In [3]:
import os

print(os.getcwd())
print(os.path.exists('../../.kaggle/16000_pcm_speeches/Benjamin_Netanyau/'))

/Users/ginovalverde/projects/deep_learning/notebooks
True


In [4]:
DATASET_ROOT = '../../.kaggle/16000_pcm_speeches'
BENJAMIN_DATA= os.path.join(DATASET_ROOT, "Benjamin_Netanyau")
JENS_DATA= os.path.join(DATASET_ROOT, "Jens_Stoltenberg")
JULIA_DATA=os.path.join(DATASET_ROOT, "Julia_Gillard")
MARGARET_DATA=os.path.join(DATASET_ROOT, "Magaret_Tarcher")
NELSON_DATA=os.path.join(DATASET_ROOT, "Nelson_Mandela")

In [5]:
for name in ["Benjamin_Netanyau", "Jens_Stoltenberg", "Julia_Gillard", "Margaret_Thatcher", "Nelson_Mandela"]:
    full_path = os.path.join(DATASET_ROOT, name)
    print(f"{name}: {full_path} -> {os.path.exists(full_path)}")

Benjamin_Netanyau: ../../.kaggle/16000_pcm_speeches/Benjamin_Netanyau -> True
Jens_Stoltenberg: ../../.kaggle/16000_pcm_speeches/Jens_Stoltenberg -> True
Julia_Gillard: ../../.kaggle/16000_pcm_speeches/Julia_Gillard -> True
Margaret_Thatcher: ../../.kaggle/16000_pcm_speeches/Margaret_Thatcher -> False
Nelson_Mandela: ../../.kaggle/16000_pcm_speeches/Nelson_Mandela -> True


In [6]:
def parse_dataset(dataset_paths):
    X=[]
    y=[]
    for index, dataset in enumerate(dataset_paths):
        print(f"Parsing dataset {dataset}...".format(dataset))
        for fname in os.listdir(dataset):
            wav,sr=librosa.load(os.path.join(dataset, fname), sr=None)
            D=librosa.amplitude_to_db(np.abs(librosa.stft(wav)), ref=np.max)
            X.append(D)
            y.append(index)
    return(X,y)

In [7]:
X,y =parse_dataset([BENJAMIN_DATA, JENS_DATA, JULIA_DATA, MARGARET_DATA, NELSON_DATA])

Parsing dataset ../../.kaggle/16000_pcm_speeches/Benjamin_Netanyau...
Parsing dataset ../../.kaggle/16000_pcm_speeches/Jens_Stoltenberg...
Parsing dataset ../../.kaggle/16000_pcm_speeches/Julia_Gillard...
Parsing dataset ../../.kaggle/16000_pcm_speeches/Magaret_Tarcher...
Parsing dataset ../../.kaggle/16000_pcm_speeches/Nelson_Mandela...


In [8]:
len(X)

7501

## Dataset split

In [9]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

In [10]:
X_test,X_val,y_test,y_val = train_test_split(X_test, y_test, test_size=0.5, random_state=42)

In [11]:
print("Longitud del subconjunto de entrenamiento: ", len(X_train))
print("Longitud del subconjunto de validación: ", len(X_val))
print("Longitud del subconjunto de prueba: ", len(X_test))

Longitud del subconjunto de entrenamiento:  6750
Longitud del subconjunto de validación:  376
Longitud del subconjunto de prueba:  375


## Model building

In [12]:
import tensorflow as tf
print(dir(tf.keras.utils))

['Config', 'CustomObjectScope', 'FeatureSpace', 'Progbar', 'PyDataset', 'Sequence', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__path__', '__spec__', 'array_to_img', 'audio_dataset_from_directory', 'bounding_boxes', 'clear_session', 'custom_object_scope', 'deserialize_keras_object', 'disable_interactive_logging', 'enable_interactive_logging', 'get_custom_objects', 'get_file', 'get_registered_name', 'get_registered_object', 'get_source_inputs', 'image_dataset_from_directory', 'img_to_array', 'is_interactive_logging_enabled', 'is_keras_tensor', 'legacy', 'load_img', 'model_to_dot', 'normalize', 'pack_x_y_sample_weight', 'pad_sequences', 'plot_model', 'register_keras_serializable', 'save_img', 'serialize_keras_object', 'set_random_seed', 'split_dataset', 'standardize_dtype', 'text_dataset_from_directory', 'timeseries_dataset_from_array', 'to_categorical', 'unpack_x_y_sample_weight']


In [13]:

from tensorflow.keras.utils import to_categorical


def prep_dataset(X,y,shape):
    X_prep=np.array(X).reshape((len(X),shape))
    X_prep=X_prep.astype('float32')/255
    y_prep=to_categorical(np.array(y))
    return(X_prep,y_prep)